In [ ]:

# # Full Meter Reading OCR Pipeline
# 
# This notebook demonstrates the complete OCR pipeline, combining detection and recognition models.

# ## 1. Environment Setup

In [ ]:

import os
import sys
import glob
import subprocess
import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Image, clear_output

# Ensure we're in the PaddleOCR root directory
ROOT_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "../.."))
os.chdir(ROOT_DIR)
print(f"Working directory: {os.getcwd()}")

# Check if key directories exist
assert os.path.exists('ppocr'), "Not in PaddleOCR root directory!"
assert os.path.exists('tools'), "PaddleOCR tools directory not found!"

# ## 2. Model Configuration

In [ ]:

# Configuration for detection and recognition models
config = {
    # Dataset paths
    "test_image_dir": "dataset/det_dataset_1/images",
    "det_model_dir": "model_training/det_train/output/meter_student",
    "rec_model_dir": "model_training/rec_train/output/meter_rec",
    "dict_path": "model_training/rec_train/meter_dict.txt",
    
    # Model selection
    "det_model_type": "student",  # "teacher" or "student"
    "rec_model_type": "standard", # "standard" or "distillation"
    
    # Config files
    "det_config": "configs/det/ch_PP-OCRv3/ch_PP-OCRv3_det_teacher.yml",
    "rec_config": "configs/rec/meter/meter_PP-OCRv3_rec.yml",
    
    # Output directory
    "output_dir": "inference_results"
}

# Adjust paths based on model types
if config["det_model_type"] == "teacher":
    config["det_model_dir"] = "model_training/det_train/output/meter_teacher"

if config["rec_model_type"] == "distillation":
    config["rec_model_dir"] = "model_training/rec_train/output/meter_rec_distillation"

# Create output directory
os.makedirs(config["output_dir"], exist_ok=True)

# Print configuration
print("Pipeline Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

# ## 3. Check Models

In [ ]:

# Check if detection model exists
det_model = os.path.join(config["det_model_dir"], "best_accuracy.pdparams")
if not os.path.exists(det_model):
    print(f"Warning: Detection model not found at {det_model}")
    print("You need to train a detection model first.")
else:
    print(f"Detection model found at {det_model}")

# Check if recognition model exists
rec_model = os.path.join(config["rec_model_dir"], "best_accuracy.pdparams")
if not os.path.exists(rec_model):
    print(f"Warning: Recognition model not found at {rec_model}")
    print("You need to train a recognition model first.")
else:
    print(f"Recognition model found at {rec_model}")

# Check character dictionary
if not os.path.exists(config["dict_path"]):
    print(f"Warning: Character dictionary not found at {config['dict_path']}")
    print("Make sure you have created the character dictionary.")
else:
    with open(config["dict_path"], 'r') as f:
        chars = [line.strip() for line in f if line.strip()]
    print(f"Character dictionary found with {len(chars)} characters: {''.join(chars)}")

# ## 4. Find Test Images

In [ ]:

# Find test images
test_images = glob.glob(os.path.join(config["test_image_dir"], "*.jpg"))
if not test_images:
    print(f"No test images found in {config['test_image_dir']}")
    # Try to find images in any subdirectory
    test_images = glob.glob(os.path.join(config["test_image_dir"], "**/*.jpg"), recursive=True)
    if not test_images:
        print("No test images found in any subdirectory.")

if test_images:
    print(f"Found {len(test_images)} test images.")
    print(f"First few images: {test_images[:3]}")
    
    # Display a sample image
    sample_image = test_images[0]
    print(f"Sample image: {sample_image}")
    display(Image(filename=sample_image))
else:
    print("Please specify a valid directory with test images.")

# ## 5. Detection Inference

In [ ]:

# Run detection inference on a sample image
def run_detection(image_path, config):
    """Run detection on a single image"""
    det_cmd = f"""python tools/infer_det.py \
        -c {config["det_config"]} \
        -o Global.infer_img={image_path} \
        Global.checkpoints={config["det_model_dir"]}/best_accuracy \
        Global.save_res_path={config["output_dir"]}/det_results.txt"""
    
    print(f"Running detection on {os.path.basename(image_path)}...")
    print(f"Command: {det_cmd}")
    
    # Uncomment to run detection
    # subprocess.run(det_cmd, shell=True)
    # 
    # # Display result if available
    # result_image = os.path.join(config["output_dir"], "det_res.jpg")
    # if os.path.exists(result_image):
    #     print("Detection result:")
    #     display(Image(filename=result_image))
    # else:
    #     print(f"Result image not found: {result_image}")
    
    return os.path.join(config["output_dir"], "det_res.jpg")

# Choose a sample image for detection
if test_images:
    sample_det_image = test_images[0]
    print(f"Sample image for detection: {sample_det_image}")
    # Uncomment to run
    # det_result = run_detection(sample_det_image, config)

# ## 6. Recognition Inference

In [ ]:

# Run recognition on cropped regions
def run_recognition(image_path, config):
    """Run recognition on a cropped image"""
    rec_cmd = f"""python tools/infer_rec.py \
        -c {config["rec_config"]} \
        -o Global.infer_img={image_path} \
        Global.checkpoints={config["rec_model_dir"]}/best_accuracy \
        Global.character_dict_path={config["dict_path"]}"""
    
    print(f"Running recognition on {os.path.basename(image_path)}...")
    print(f"Command: {rec_cmd}")
    
    # Uncomment to run recognition
    # process = subprocess.Popen(rec_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    # output = ""
    # for line in iter(process.stdout.readline, ''):
    #     print(line, end='')
    #     output += line
    # process.wait()
    
    # # Parse and return recognition result
    # # This is a simple parsing, adjust based on actual output format
    # if "result" in output:
    #     result_line = [l for l in output.split('\n') if "result" in l]
    #     if result_line:
    #         return result_line[0].split("result: ")[1].strip()
    
    return "Recognition result would appear here"

# For demonstration, we would typically run this on detected regions
# But for simplicity, we'll just use a sample test image
if test_images:
    sample_rec_image = test_images[0]
    print(f"Sample image for recognition: {sample_rec_image}")
    # Uncomment to run
    # rec_result = run_recognition(sample_rec_image, config)
    # print(f"Recognition result: {rec_result}")

# ## 7. Full OCR Pipeline

In [ ]:

# Run complete OCR pipeline on an image
def run_full_ocr(image_path, config):
    """Run full OCR pipeline (detection + recognition) on an image"""
    ocr_cmd = f"""python tools/infer.py \
        -c {config["det_config"]} \
        -o Global.infer_img={image_path} \
        Global.use_gpu=true \
        Global.det_algorithm=DB \
        Global.det_model_dir={config["det_model_dir"]} \
        Global.rec_algorithm=SVTR_LCNet \
        Global.rec_model_dir={config["rec_model_dir"]} \
        Global.rec_char_dict_path={config["dict_path"]} \
        Global.save_res_path={config["output_dir"]}/ocr_result.txt \
        Global.vis_font_path=./doc/fonts/simfang.ttf"""
    
    print(f"Running full OCR pipeline on {os.path.basename(image_path)}...")
    print(f"Command: {ocr_cmd}")
    
    # Uncomment to run the full pipeline
    # subprocess.run(ocr_cmd, shell=True)
    # 
    # # Display result if available
    # result_image = image_path.replace(os.path.basename(image_path), "inference_results")
    # result_image = os.path.join(os.path.dirname(result_image), os.path.basename(image_path))
    # if os.path.exists(result_image):
    #     print("OCR result:")
    #     display(Image(filename=result_image))
    # else:
    #     print(f"Result image not found: {result_image}")
    #     # Try alternative path
    #     result_image = os.path.join(config["output_dir"], os.path.basename(image_path))
    #     if os.path.exists(result_image):
    #         print("OCR result:")
    #         display(Image(filename=result_image))
    
    # Display the result text file if available
    result_file = os.path.join(config["output_dir"], "ocr_result.txt")
    if os.path.exists(result_file):
        with open(result_file, 'r') as f:
            results = f.read()
        print("OCR results:")
        print(results)
    else:
        print(f"Result file not found: {result_file}")

# Choose a sample image for full OCR
if test_images:
    sample_ocr_image = test_images[0]
    print(f"Sample image for full OCR: {sample_ocr_image}")
    # Uncomment to run
    # run_full_ocr(sample_ocr_image, config)

# ## 8. Batch Processing

In [ ]:

# Process multiple images in batch
def batch_process(image_list, config, max_images=5):
    """Process multiple images with full OCR pipeline"""
    if not image_list:
        print("No images provided for batch processing.")
        return
    
    # Limit number of images for demonstration
    if len(image_list) > max_images:
        print(f"Limiting to {max_images} images for demonstration.")
        image_list = image_list[:max_images]
    
    print(f"Processing {len(image_list)} images...")
    
    results = []
    for image_path in image_list:
        print(f"Processing {os.path.basename(image_path)}...")
        
        # Run OCR pipeline
        ocr_cmd = f"""python tools/infer.py \
            -c {config["det_config"]} \
            -o Global.infer_img={image_path} \
            Global.use_gpu=true \
            Global.det_algorithm=DB \
            Global.det_model_dir={config["det_model_dir"]} \
            Global.rec_algorithm=SVTR_LCNet \
            Global.rec_model_dir={config["rec_model_dir"]} \
            Global.rec_char_dict_path={config["dict_path"]} \
            Global.save_res_path={config["output_dir"]}/batch_results/{os.path.basename(image_path)}.txt"""
        
        # Create output directory
        os.makedirs(os.path.join(config["output_dir"], "batch_results"), exist_ok=True)
        
        # Uncomment to run
        # subprocess.run(ocr_cmd, shell=True)
        # 
        # # Read results
        # result_file = f"{config['output_dir']}/batch_results/{os.path.basename(image_path)}.txt"
        # if os.path.exists(result_file):
        #     with open(result_file, 'r') as f:
        #         ocr_result = f.read()
        #     results.append((image_path, ocr_result))
        # else:
        #     results.append((image_path, "No result file found"))
    
    # Print summary
    print("\nBatch processing summary:")
    for image_path, result in results:
        print(f"Image: {os.path.basename(image_path)}")
        print(f"Result: {result[:100]}..." if len(result) > 100 else f"Result: {result}")
        print("-" * 40)

# Choose images for batch processing
if test_images:
    batch_images = test_images[:3]  # Take first 3 images for example
    print(f"Selected {len(batch_images)} images for batch processing:")
    for img in batch_images:
        print(f"  - {os.path.basename(img)}")
    
    # Uncomment to run batch processing
    # batch_process(batch_images, config)

# ## 9. Model Deployment

In [ ]:

# Export models for deployment
def export_models(config):
    """Export detection and recognition models for deployment"""
    # Export detection model
    det_export_cmd = f"""python tools/export_model.py \
        -c {config["det_config"]} \
        -o Global.pretrained_model={config["det_model_dir"]}/best_accuracy \
        Global.save_inference_dir={config["det_model_dir"]}/inference"""
    
    # Export recognition model
    rec_export_cmd = f"""python tools/export_model.py \
        -c {config["rec_config"]} \
        -o Global.pretrained_model={config["rec_model_dir"]}/best_accuracy \
        Global.save_inference_dir={config["rec_model_dir"]}/inference \
        Global.character_dict_path={config["dict_path"]}"""
    
    print("Detection model export command:")
    print(det_export_cmd)
    
    print("\nRecognition model export command:")
    print(rec_export_cmd)
    
    # Uncomment to run export
    # print("\nExporting detection model...")
    # subprocess.run(det_export_cmd, shell=True)
    # 
    # print("\nExporting recognition model...")
    # subprocess.run(rec_export_cmd, shell=True)
    # 
    # # Check if export was successful
    # if os.path.exists(os.path.join(config["det_model_dir"], "inference", "inference.pdmodel")):
    #     print("Detection model exported successfully.")
    # else:
    #     print("Detection model export failed.")
    # 
    # if os.path.exists(os.path.join(config["rec_model_dir"], "inference", "inference.pdmodel")):
    #     print("Recognition model exported successfully.")
    # else:
    #     print("Recognition model export failed.")

# Show export commands
export_models(config)

# ## 10. Performance Analysis

In [ ]:

# Analyze performance of OCR system
def analyze_performance(test_images, config, max_images=5):
    """Analyze performance on test images with ground truth"""
    if not test_images:
        print("No test images provided for analysis.")
        return
    
    # Limit number of images for demonstration
    if len(test_images) > max_images:
        print(f"Limiting to {max_images} images for demonstration.")
        test_images = test_images[:max_images]
    
    # Create a table for results
    results = []
    header = ["Image", "Detection Time (ms)", "Recognition Time (ms)", "Total Time (ms)", "Text Detected"]
    
    # Uncomment to run performance analysis
    # for image_path in test_images:
    #     print(f"Processing {os.path.basename(image_path)}...")
    #     
    #     # Measure detection time
    #     start_time = time.time()
    #     det_cmd = f"""python tools/infer_det.py \
    #         -c {config["det_config"]} \
    #         -o Global.infer_img={image_path} \
    #         Global.checkpoints={config["det_model_dir"]}/best_accuracy"""
    #     subprocess.run(det_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #     det_time = (time.time() - start_time) * 1000  # Convert to ms
    #     
    #     # Get detection result
    #     det_result_file = os.path.join(config["output_dir"], "det_results.txt")
    #     det_boxes = []
    #     if os.path.exists(det_result_file):
    #         with open(det_result_file, 'r') as f:
    #             det_data = f.read()
    #             # Parse detection boxes (format depends on output)
    #             # This is placeholder parsing, adjust based on actual format
    #             det_boxes = [line.split('\t')[0] for line in det_data.split('\n') if line]
    #     
    #     # Measure recognition time
    #     start_time = time.time()
    #     # We'll use the full image for simplicity
    #     rec_cmd = f"""python tools/infer_rec.py \
    #         -c {config["rec_config"]} \
    #         -o Global.infer_img={image_path} \
    #         Global.checkpoints={config["rec_model_dir"]}/best_accuracy \
    #         Global.character_dict_path={config["dict_path"]}"""
    #     proc = subprocess.run(rec_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    #     rec_time = (time.time() - start_time) * 1000  # Convert to ms
    #     
    #     # Extract recognized text
    #     rec_output = proc.stdout
    #     recognized_text = "Unknown"
    #     # Parse recognition result (format depends on output)
    #     # This is placeholder parsing, adjust based on actual format
    #     if "result" in rec_output:
    #         for line in rec_output.split('\n'):
    #             if "result" in line:
    #                 recognized_text = line.split("result: ")[1].strip()
    #                 break
    #     
    #     # Add to results
    #     total_time = det_time + rec_time
    #     results.append([os.path.basename(image_path), f"{det_time:.1f}", f"{rec_time:.1f}", f"{total_time:.1f}", recognized_text])
    # 
    # # Display results table
    # from tabulate import tabulate
    # print(tabulate(results, headers=header, tablefmt="grid"))
    
    # This is just a placeholder for demonstration
    print("Performance analysis would analyze:")
    for img in test_images:
        print(f"  - {os.path.basename(img)}")
    print("\nMetrics would include:")
    print("  - Detection time")
    print("  - Recognition time")
    print("  - Total processing time")
    print("  - Text detection accuracy")
    print("  - Text recognition accuracy")

# Run performance analysis if test images are available
if test_images:
    analyze_performance(test_images, config) 